In [1]:
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm
#os.environ['CUDA_VISIBLE_DEVICES'] = '3, 5, 7'
from config_cls import cfg
from dataset import Dataset
from factory_cls import backbone
from random import randint
from tensorflow.python.framework import graph_util
import pandas as pd

class Classitrain(object):
    def __init__(self):
        self.Batch_Size = cfg.TRAIN.BATCHSIZE
        self.learn_rate_init = cfg.TRAIN.LEARN_RATE_INIT
        self.num_classes = cfg.TRAIN.NUMCLASS
        self.trainset = Dataset(self.num_classes,'train')
        self.testset = Dataset(self.num_classes,'test')
        self.network = cfg.TRAIN.NETWORK
        self.train_txt = cfg.TRAIN.ANNO_PATH
        self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
        self.model_type = cfg.TRAIN.NETWORK
        self.input_size = cfg.TRAIN.INPUTSIZE
        self.interval = cfg.TRAIN.SAVE
        self.initial_weights = cfg.TRAIN.INITIAL_WEIGHT
        self.pretrain_mode = cfg.TRAIN.PRETRAIN_MODE
        self.epoch = cfg.TRAIN.EPOCH
        self.pretrain_model = cfg.TRAIN.BACKBONE_PRETRAIN
        self.moving_ave_decay = cfg.TRAIN.MOMENTUM
        self.steps_per_period  = len(self.trainset)
        self.learn_rate_end = cfg.TRAIN.LEARN_RATE_END
        self.quantity = len(open(self.train_txt, 'r').readlines())
        self.keep_prob = tf.placeholder(dtype=tf.float32, name='dropout')
        self.moving_ave_decay = 0.995
        self.warmup_periods = 1
        self.gpu_list = [3, 7]
       
        self.input_data = tf.placeholder(shape = [None, self.input_size, self.input_size, 3], dtype=tf.float32, name='input')
        self.input_labels = tf.placeholder(dtype=tf.float32, name='label')
        self.trainable = tf.placeholder(dtype=tf.bool, name='trainable')

        
        #self.model = backbone(model=self.network, input_data=self.input_data, trainable=self.trainable, classes=self.num_classes, keep_prob=self.keep_prob)
        #self.loss, self.accurate = self.model.compute_loss(labels=self.input_labels)
        #self.cam = self.model.cam()
#         self.net_var = tf.global_variables()
#         self.varaibles_to_restore = [var for var in self.net_var if 'backbone' in var.name]
        
            
        #moving_ave = tf.train.ExponentialMovingAverage(self.moving_ave_decay).apply(tf.trainable_variables())
        with tf.name_scope('learn_rate'):
            self.global_step = tf.Variable(1.0, dtype=tf.float64, trainable=False, name='global_step')
            warmup_steps = tf.constant(self.warmup_periods * self.steps_per_period,
                                        dtype=tf.float64, name='warmup_steps')
            train_steps = tf.constant( self.epoch * self.steps_per_period,
                                        dtype=tf.float64, name='train_steps')
            self.learn_rate = tf.cond(
                pred=self.global_step < warmup_steps,
                true_fn=lambda: self.global_step / warmup_steps * self.learn_rate_init,
                false_fn=lambda: self.learn_rate_end + 0.5 * (self.learn_rate_init - self.learn_rate_end) *
                                    (1 + tf.cos(
                                        (self.global_step - warmup_steps) / (train_steps - warmup_steps) * np.pi))
            )
            self.global_step_update = tf.assign_add(self.global_step, 1.0)
            
        #moving_ave = tf.train.ExponentialMovingAverage(self.moving_ave_decay).apply(tf.trainable_variables())
        #self.optimizer =  tf.train.AdamOptimizer(self.learn_rate).minimize(self.loss, var_list=self.net_var)
        self.optimizer =  tf.train.AdamOptimizer(self.learn_rate)
        
        tower_grads = []
        tower_loss = []
        tower_acc = []
        for d in self.gpu_list:
            with tf.device('/GPU:%d' %d):
                print('tower:%d...'% d)
                with tf.name_scope('tower_%s' %d):
                    with tf.variable_scope('whole_network', reuse=int(d) > int(self.gpu_list[0])):
                        model = backbone(model=self.network, input_data=self.input_data, trainable=self.trainable, classes=self.num_classes, keep_prob=self.keep_prob)
                        loss, accurate = model.compute_loss(labels=self.input_labels)
                        grads = self.optimizer.compute_gradients(loss)
                        tower_grads.append(grads)
                        tower_loss.append(loss)
                        tower_acc.append(accurate)
                        tf.get_variable_scope().reuse_variables()
        self.mean_loss = tf.reduce_mean(tower_loss)
        mean_grads = self.average_gradients(tower_grads)
        self.mean_acc = tf.reduce_mean(tower_acc)
            #self.optimizer = tf.train.MomentumOptimizer(learning_rate=self.learn_rate_init, momentum=0.8).minimize(self.loss)
            
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
                with tf.control_dependencies([self.global_step_update]):
                    self.train_op = self.optimizer.apply_gradients(mean_grads)
        
        self.net_var = tf.global_variables()
        self.varaibles_to_restore = [var for var in self.net_var if 'backbone' in var.name]
        self.loader_backbone = tf.train.Saver(self.varaibles_to_restore)
        self.loader_whole = tf.train.Saver(tf.global_variables())
        self.saver = tf.train.Saver(tf.global_variables(), max_to_keep=10)
    
    
    
    def feed_all_gpu(self, inp_dict, dataset):
        pay_load_per_gpu = self.Batch_Size / len(self.gpu_list)
        for i in range(len(self.gpu_list)):
            start_pos = i * pay_load_per_gpu
            stop_pos = (i+1) * pay_load_per_gpu
            inp_dict[self.input_data] = dataset[0][start_pos:stop_pos, ...]
            inp_dict[self.input_labels] = dataset[1][start_pos:stop_pos, ...]
        return inp_dict
        
        
    def average_gradients(self, tower_grads):
        average_grads = []
        for grad_and_vars in zip(*tower_grads):
            # Note that each grad_and_vars looks like the following:
            #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
            grads = [g for g, _ in grad_and_vars]
            # Average over the 'tower' dimension.
            grad = tf.stack(grads, 0)
            grad = tf.reduce_mean(grad, 0)

            # Keep in mind that the Variables are redundant because they are shared
            # across towers. So .. we will just return the first tower's pointer to
            # the Variable.
            v = grad_and_vars[0][1]
            grad_and_var = (grad, v)
            average_grads.append(grad_and_var)

        return average_grads


    def train(self):
        #self.sess.run(tf.global_variables_initializer())
        if self.pretrain_mode == 'whole':
            try:
                print ('=>Restore weights from ' + self.initial_weights)
                self.loader_whole.restore(self.sess, self.initial_weights)
            except:
                print (self.initial_weights + 'does not exist!')
                print ('=>starts training from scratch ...')
        else:
            try:
                print ('=>Restore weights from ' + self.pretrain_model)
                self.loader_backbone.restore(self.sess, self.pretrain_model)
            except:
                print (self.pretrain_model + 'does not exist!')
                print ('=>starts training from scratch ...')
        
        min_loss_val = 0.8
        min_loss_train = 0.8
        i = 0
        for epoch in range(self.epoch):
            pabr = tqdm(self.trainset)
            train_epoch_loss, test_epoch_loss = [], []
            train_epoch_acc, test_epoch_acc = [], []
            inp_dict = {}
            inp_dict[self.trainable] = True
            inp_dict[self.keep_prob] = 1.0
            for train_dataset in pabr:
                inp_dict = self.feed_all_gpu(inp_dict, train_dataset)
                _, train_step_loss, train_step_acc, _ = self.sess.run([self.train_op, self.mean_loss, self.mean_acc, self.global_step], 
                                                                      feed_dict=inp_dict)
                train_epoch_loss.append(train_step_loss)
                train_epoch_acc.append(train_step_acc)
                pabr.set_description("train loss: %.2f" %train_step_loss)
            
            inp_dict = {}
            inp_dict[self.trainable] = False
            inp_dict[self.keep_prob] = 1.0
            for test_data in self.testset:
                inp_dict = self.feed_all_gpu(inp_dict, test_data)
                test_step_loss, test_step_acc = self.sess.run([self.mean_loss, self.mean_acc],
                                                                feed_dict=inp_dict)

                test_epoch_loss.append(test_step_loss)
                test_epoch_acc.append(test_step_acc)

            train_epoch_loss, train_epoch_acc, test_epoch_loss, test_epoch_acc = np.mean(train_epoch_loss), np.mean(train_epoch_acc), np.mean(test_epoch_loss), np.mean(test_epoch_acc)
            print ('Epoch: %2d Train loss: %.2f Train acc: %.2f'
                   %(epoch, train_epoch_loss, train_epoch_acc))

            print ('Test loss: %.2f Test acc: %.2f'
                   % (test_epoch_loss, test_epoch_acc))
            
            
            if epoch >= 10 and test_epoch_loss < min_loss_val and train_epoch_loss < min_loss_train:
                min_loss_val = test_epoch_loss
                min_loss_train = train_epoch_loss
#                 constant_graph = graph_util.convert_variables_to_constants(self.sess, self.sess.graph_def, ['fc_layer/op_to_store', 'cam'])
#                 model_name = self.model_type+'_epoch=%d' %epoch
#                 ckpt_file = '/hdd/sd5/tlc/TCT/Model_ckpt/'+ model_name + '_test_loss=%.4f.ckpt' %test_epoch_loss
#                 self.saver.save(self.sess, ckpt_file, global_step=epoch)
#                 with tf.gfile.FastGFile('/hdd/sd5/tlc/TCT/Model_pb/'+model_name+'.pb', mode='wb') as f:
#                     f.write(constant_graph.SerializeToString())


                

    def main(self):
        self.sess.run(tf.global_variables_initializer())
        self.train()
        
if __name__ == '__main__':
    Classitrain().main() 





Instructions for updating:
Colocations handled automatically by placer.
tower:3...
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use tf.cast instead.
tower:7...
=>Restore weights from /hdd/sd5/tlc/pretrain_model/darknet53/yolov3_test_loss=136.5372.ckpt-80
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /hdd/sd5/tlc/pretrain_model/darknet53/yolov3_test_loss=136.5372.ckpt-80


  0%|          | 0/338 [00:00<?, ?it/s]

/hdd/sd5/tlc/pretrain_model/darknet53/yolov3_test_loss=136.5372.ckpt-80does not exist!
=>starts training from scratch ...


  0%|          | 0/338 [00:00<?, ?it/s]

Epoch:  0 Train loss: 0.82 Train acc: 0.62
Test loss: 0.83 Test acc: 0.28


  0%|          | 0/338 [00:00<?, ?it/s]

Epoch:  1 Train loss: 0.81 Train acc: 0.67
Test loss: 0.85 Test acc: 0.73


train loss: 0.81:  38%|███▊      | 128/338 [18:07<26:52,  7.68s/it]ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/miniconda3/envs/py2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/opt/miniconda3/envs/py2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/opt/miniconda3/envs/py2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/opt/miniconda3/envs/py2/lib/python2.7/inspect.py", line 1058, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/opt/miniconda3/envs/py2/lib/python2.7/inspect.py", line 1018, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/opt/miniconda3/envs/py2/lib/python2.7/inspect.py", line 453, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader

IndexError: string index out of range

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
img = np.load('/ssd2/yuyue/TCT_data/20200515_data/train/hard_224_neg/500_data/6_cut/570ba122b419486ac24a3a1e5b87bd673794f45b_000209_3.npy')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray.shape

(224, 224)